<a href="https://colab.research.google.com/github/jaredlan1/getting_started_with_ML/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt


In [2]:
# -------- activation functions -------
def relu(z):
    if z > 0:
        return z
    else:
        return 0

def relu_back(xbar, z):
    if z > 0:
        return xbar
    else:
        return 0

identity = lambda z: z

identity_back = lambda xbar, z: xbar
# -------------------------------------------

In [3]:
# ---------- initialization -----------
def initialization(nin, nout):
    W = np.random.randn(nout, nin)# /np.sqrt(nin)
    b = np.zeros((nout, 1))
    return W, b
# -------------------------------------


# -------- loss functions -----------
def mse(yhat, y):
    diff = yhat-y
    sq = diff**2
    mse = np.mean(sq)
    return mse

def mse_back(yhat, y):

    # derivative of y of mse
    diff = yhat-y
    n = y.shape[0]
    return  2*(diff) / n
# -----------------------------------

In [5]:
# ------------- Layer ------------
class Layer:
    def __init__(self, nin, nout, activation=identity):
        self.W = np.random.rand(nin, nout)
        self.b = np.zeros((1, nout))

        self.activation = activation
        if activation == relu:
            self.activation_back = relu_back
        if activation == identity:
            self.activation_back = identity_back

        self.Wbar = None
        self.bbar = None

        self.X_cache = None
        self.Z_cache = None

    def forward(self, X, train=True):
        Z = X @ self.W + self.b
        Xnew = self.activation(Z)

        if train:
            self.X_cache = X
            self.Z_cache = Z

        return Xnew

    def backward(self, Xnewbar):
        Zbar = self.activation_back(Xnewbar, self.Z_cache)

        self.Wbar = self.X_cache.T @ Zbar
        self.bbar = np.sum(Zbar, axis=0, keepdims=True)

        Xbar = Zbar @ self.W.T
        return Xbar

In [ ]:
class Network:
    def __init__(self, layers, loss):
        self.layers = layers

        if loss == mse:
            self.loss_back = mse_back

        # caches
        self.yhat_cache = None
        self.y_cache = None

    def forward(self, X, y, train=True):
        # propagate input through layers
        for layer in self.layers:
            X = layer.forward(X, train=train)

        yhat = X
        L = mse(yhat, y)

        if train:
            self.yhat_cache = yhat
            self.y_cache = y

        return L, yhat

    def backward(self):
        # start from loss gradient
        Lbar = self.loss_back(self.yhat_cache, self.y_cache)

        # backprop through layers
        for layer in reversed(self.layers):
            Lbar = layer.backward(Lbar)



In [ ]:
class GradientDescent:

    def __init__(self, alpha):
        self.alpha = alpha

    def step(self, network):
        for layer in network.layer

          layer.W -= self.alpha * layer.Wbar
          layer.b -= self..alpha * layerbbar


if __name__ == '__main__':

    # ---------- data preparation ----------------
    # Initialize lists for the numeric data and the string data
    numeric_data = []

    # Read the text file
    with open('auto-mpg.data', 'r') as file:
        for line in file:
            # Split the line into columns
            columns = line.strip().split()

            # Check if any of the first 8 columns contain '?'
            if '?' in columns[:8]:
                continue  # Skip this line if there's a missing value

            # Convert the first 8 columns to floats and append to numeric_data
            numeric_data.append([float(value) for value in columns[:8]])

    # Convert numeric_data to a numpy array for easier manipulation
    numeric_array = np.array(numeric_data)


In [ ]:
# Shuffle the numeric array and the corresponding string array
    nrows = numeric_array.shape[0]
    indices = np.arange(nrows)
    np.random.shuffle(indices)
    shuffled_numeric_array = numeric_array[indices]

    # Split into training (80%) and test (20%) sets
    split_index = int(0.8 * nrows)

    train_numeric = shuffled_numeric_array[:split_index]
    test_numeric = shuffled_numeric_array[split_index:]

    # separate inputs/outputs
    Xtrain = train_numeric[:, 1:]
    ytrain = train_numeric[:, 0]

    Xtest = test_numeric[:, 1:]
    ytest = test_numeric[:, 0]

    # normalize
    Xmean = np.mean(Xtrain, axis=0)
    Xstd = np.std(Xtrain, axis=0)
    ymean = np.mean(ytrain)
    ystd = np.std(ytrain)

    Xtrain = (Xtrain - Xmean) / Xstd
    Xtest = (Xtest - Xmean) / Xstd
    ytrain = (ytrain - ymean) / ystd
    ytest = (ytest - ymean) / ystd

    # reshape arrays (opposite order of pytorch, here we have nx x ns).
    # I found that to be more conveient with the way I did the math operations, but feel free to setup
    # however you like.
    Xtrain = Xtrain.T
    Xtest = Xtest.T
    ytrain = np.reshape(ytrain, (1, len(ytrain)))
    ytest = np.reshape(ytest, (1, len(ytest)))

    # ------------------------------------------------------------

    l1 = Layer(7, ?, relu)
    # TODO
    layers = [l1, l2, l3]
    network = Network(layers, mse)
    alpha = ?
    optimizer = GradientDescent(alpha)

    train_losses = []
    test_losses = []
    epochs = ?
    for i in range(epochs):
        # TODO: run train set, backprop, step

        # TODO: run test set


    # --- inference ----
    _, yhat = network.forward(Xtest, ytest, train=False)

    # unnormalize
    yhat = (yhat * ystd) + ymean
    ytest = (ytest * ystd) + ymean

    plt.figure()
    plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
    plt.plot(range(1, epochs + 1), test_losses, label='Testing Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Testing Losses')
    plt.legend()


    plt.figure()
    plt.plot(ytest.T, yhat.T, "o")
    plt.plot([10, 45], [10, 45], "--")

    print("avg error (mpg) =", np.mean(np.abs(yhat - ytest)))

    plt.show()